In [1]:
# from datasets import load_dataset

import os
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Qdrant
from langchain.text_splitter import RecursiveCharacterTextSplitter
from qdrant_client import QdrantClient
from langchain.schema import Document
from qdrant_client.http.models import Distance, VectorParams
from langchain_qdrant import QdrantVectorStore


import pandas as pd
pd.set_option('display.max_columns', None)

In [2]:
# ds = load_dataset("IlyaGusev/habr", trust_remote_code=True)
# ds.save_to_disk("D:\Vscode_projects\Intro_to_LLM\data\habr_data")
# df = ds["train"].to_pandas()

In [3]:
df10 = pd.read_csv('../data/habr_10_percent.csv')


In [4]:
df10.sample(3)

,id,language,url,title,text_markdown,text_html,author,original_author,original_url,lead_html,lead_markdown,type,time_published,statistics,labels,hubs,flows,tags,reading_time,format,complexity,comments
27781,264573,ru,https://habr.com/ru/post/264573/,Как не нужно строить сети,Доброго времени суток.\nНа самом деле никогда ...,"<div xmlns=""http://www.w3.org/1999/xhtml""><img...",ProotIK,NaN,NaN,"<img src=""https://habrastorage.org/getpro/habr...",Доброго времени суток.\nНа самом деле никогда ...,article,1440054392,"{'commentsCount': 14, 'favoritesCount': 465, '...",[],['itstandarts'],['develop'],['сети передачи данных'],9.0,NaN,NaN,"{'id': array([8543031, 8543039, 8543077, 85431..."
5281,431338,ru,https://habr.com/ru/post/431338/,Книга «ДНК. История генетической революции»,Познакомьтесь с историей генетической революци...,"<div xmlns=""http://www.w3.org/1999/xhtml""><a h...",ph_piter,NaN,NaN,"<a href=""https://habr.com/company/piter/blog/4...",Познакомьтесь с историей генетической революци...,article,1543395788,"{'commentsCount': 4, 'favoritesCount': 34, 're...",[],['piter' 'read' 'popular_science' 'brain'],['management' 'popsci'],['книги'],14.0,NaN,NaN,"{'id': array([19425684, 19425780, 19425828, 19..."
27017,508946,ru,https://habr.com/ru/post/508946/,"Руководство по анализу Sysmon-угроз, часть 1",Эта статья является первой частью серии по ана...,"<div xmlns=""http://www.w3.org/1999/xhtml""><img...",Лада Розенкранц (Varonis),Andy Green,https://www.varonis.com/blog/sysmon-threat-det...,"<img src=""https://habrastorage.org/webt/5z/nu/...",Эта статья является первой частью серии по ана...,article,1593520560,"{'commentsCount': 2, 'favoritesCount': 22, 're...",['translation'],['varonis' 'infosecurity' 'sys_admin' 'powersh...,['develop' 'admin'],['sysmon' 'powershell' 'microsoft' 'графы угроз'],6.0,NaN,NaN,"{'id': array([21824224, 21824682], dtype=uint6..."


In [5]:
df10['text_w_title'] = df10['title'] + "\n" +df10['text_markdown']
df10["text_w_title"] = df10["text_w_title"].astype(str)
df10["url"] = df10["url"].astype(str)
df10["title"] = df10["title"].astype(str)

## Создание векторной БД и добавление туда документов

In [6]:
separators=[
        "\n\n",             # Разделение по абзацам
        "\n",               # Перенос строки
        "[code]", "```",    # Кодовые блоки
        "####",             # Заголовки четвертого уровня
        "###",              # Заголовки третьего уровня
        "##",               # Заголовки второго уровня
        "#",                # Заголовки первого уровня
        ".",                # Точки в предложениях
        " "
    ]
def create_doc_list(df10, chunk_size=1024, chunk_overlap=128, separators=separators):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, separators=separators,
        chunk_overlap=chunk_overlap)
    
    documents = []
    for _, row in df10.iterrows():
        chunks = text_splitter.split_text(row["text_w_title"])
        for chunk in chunks:
            documents.append(Document(
                page_content=chunk,
                metadata={"link": str(row["url"]), "title": str(row["title"])}
            ))
    return documents

In [7]:
# docs = create_doc_list(df10)
# embedder = HuggingFaceEmbeddings(model_name="sergeyzh/rubert-tiny-turbo")
# client = QdrantClient(path="../data/langchain_qdrant_10percent")

# client.create_collection(
#     collection_name="embed_collection_10percent",
#     vectors_config=VectorParams(size=312, distance=Distance.COSINE),
# )

# vector_store = QdrantVectorStore(
#     client=client,
#     collection_name="embed_collection_10percent",
#     embedding=embedder,
# )

# vector_store.add_documents(docs)

### Подключение к Qdrant

In [ ]:
# vector_store = QdrantVectorStore(
#     client=client,
#     collection_name="embed_collection_10percent",
#     embedding=embedder,
# )

In [2]:
client = QdrantClient(path="../data/langchain_qdrant_10percent")
embedder = HuggingFaceEmbeddings(model_name="sergeyzh/rubert-tiny-turbo")
collection_name = "embed_collection_10percent"

vector_store = QdrantVectorStore(
    client=client,
    collection_name=collection_name,
    embedding=embedder
)

C:\Users\nikit\AppData\Local\Temp\ipykernel_17112\2378406736.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedder = HuggingFaceEmbeddings(model_name="sergeyzh/rubert-tiny-turbo")
d:\Programming\RAG_Habr\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k":5})

In [4]:
query = "Хочешь в университет — сделай презентацию\nАбитуриенты, поступающие в Университет Чикаго, уже начиная с этой осени, будут обязаны, помимо сдачи основных экзаменов, представить четырехстраничные презентации в формате PowerPoint — на свободную тему.\nЧастично новые требования связаны с тем, что PowerPoint становится одним из бизнес-инструментов, а кроме того, считают авторы инициативы, в презентации абитуриент сможет раскрыть свои новые качества, которые не смогут проявиться на традиционных экзаменах.\nФормат PowerPoint стал своего рода языком делового общения. Ежедневно, по оценкам Microsoft, в мире показывается порядка 30 млн презентаций.\nvia AP"
results = retriever.get_relevant_documents(query)

for result in results:
    print(result)
    print("\n")


C:\Users\nikit\AppData\Local\Temp\ipykernel_14880\2087968362.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = retriever.get_relevant_documents(query)


page_content='Хочешь в университет — сделай презентацию
Абитуриенты, поступающие в Университет Чикаго, уже начиная с этой осени, будут обязаны, помимо сдачи основных экзаменов, представить четырехстраничные презентации в формате PowerPoint — на свободную тему.
Частично новые требования связаны с тем, что PowerPoint становится одним из бизнес-инструментов, а кроме того, считают авторы инициативы, в презентации абитуриент сможет раскрыть свои новые качества, которые не смогут проявиться на традиционных экзаменах.
Формат PowerPoint стал своего рода языком делового общения. Ежедневно, по оценкам Microsoft, в мире показывается порядка 30 млн презентаций.
via AP' metadata={'link': 'https://habr.com/ru/post/12730/', 'title': 'Хочешь в университет — сделай презентацию', '_id': '764af7ce1e064141a8f4b541ee452472', '_collection_name': 'embed_collection_10percent'}


page_content='Магистратура для стартапов
Презентация магистерской программы «Master of Technology Entrepreneurship» начнется в 18.00

In [4]:
from transformers import AutoTokenizer
from vllm import LLM, SamplingParams
from langchain.prompts import PromptTemplate

model_name = "t-tech/T-lite-it-1.0" # Указание устройства CPU
llm = LLM(model=model_name, max_model_len=8192, device="cpu")
sampling_params = SamplingParams(temperature=0.7, repetition_penalty=1.05, top_p=0.8, top_k=70)

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

prompt = PromptTemplate(
    input_variables=["system_prompt", "context", "question"],
    template="System: {system_prompt}\nContext: {context}\nQuestion: {question}"
)

def generate_answer(query, retriever):
    relevant_docs = retriever.get_relevant_documents(query)
    context = "\n".join([doc.page_content for doc in relevant_docs])
    system_prompt = "Ты T-lite, виртуальный ассистент в Т-Технологии. Твоя задача - быть полезным диалоговым ассистентом."
    input_text = prompt.format(
        system_prompt=system_prompt,
        context=context,
        question=query
    )


    prompt_token_ids = tokenizer(input_text, return_tensors="pt").input_ids

    outputs = llm.generate(prompt_token_ids=prompt_token_ids, sampling_params=sampling_params)
    generated_text = outputs[0].outputs[0].text

    return generated_text


queries = ["Что такое FAISS?", "Как работает GPT?", "Что такое векторные базы данных?"]

for query in tqdm(queries, desc="Processing queries"):
    answer = generate_answer(query, retriever)
    print(f"Вопрос: {query}")
    print(f"Ответ: {answer}\n")


KeyboardInterrupt: 

In [ ]:
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationChain

In [ ]:
class BotWithSummary:
    def __init__(self, llm):
        self.memory = ConversationSummaryMemory(llm=llm)

        self.conversation = ConversationChain(
            llm=llm,
            verbose=False,
            memory=self.memory
        )
        
    def _respond(self, user_input):
        # пихаем функцию для генерации ответа и проверки ее SELF RAGом
        return self.conversation.predict(input=user_input)
        
    def print_memory(self):
        print("Memory content:")
        print(self.memory.load_memory_variables({})) 

        
    def run(self):
        print("Habr chat bot:")
        while True:
            user_input = input("User: ")
            if user_input == "Ещкере":
                break
            print(f"User: {user_input}")
            try:
                response = self._respond(user_input)
                print(f"Bot: {response}")
            except Exception as e:
                print(f"Error: {str(e)}")
            time.sleep(0.2)

In [ ]:
chat = BotWithSummary(giga)
chat.run()
chat.print_memory()

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
#from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_history_aware_retriever

In [ ]:
# contextualize_q_system_prompt = (
#     "Учитывая историю чатов и последний вопрос пользователя "
#     "который может ссылаться на контекст в истории чата, "
#     "сформулируй отдельный вопрос, который может быть понят "
#     "без истории чата. НЕ отвечайте на вопрос, "
#     "только переформулируйте его, если нужно, а в противном случае верни его как есть."
# )
# contextualize_q_prompt = ChatPromptTemplate.from_messages(
#     [
#         ("system", contextualize_q_system_prompt),
#         MessagesPlaceholder("chat_history"),
#         ("human", "{input}"),
#     ]
# )
# history_aware_retriever = create_history_aware_retriever(
#     llm, retriever, contextualize_q_prompt
# )


# system_prompt = (
#     "Ты - помощник для решения задач, связанных с ответами на вопросы. "
#     "Используй следующие фрагменты извлеченного контекста, чтобы ответить на вопрос. "
#     "Если ты не знаешь ответа, скажи, что ты "
#     "не знаешь."
#     "\n\n"
#     "{context}"
# )
# qa_prompt = ChatPromptTemplate.from_messages(
#     [
#         ("system", system_prompt),
#         MessagesPlaceholder("chat_history"),
#         ("human", "{input}"),
#     ]
# )


# question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

# rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)





# system_prompt = (
#         "Ты - помощник для решения задач, связанных с ответами на вопросы. "
#         "Учитывая историю чатов и последний вопрос пользователя, "
#         "который может ссылаться на контекст в истории чата, "
#         "ответь на этот вопрос ИСПОЛЬЗУЯ фрагменты извлеченного контекста. "
#         "Если ты не знаешь ответа, скажи, что ты не знаешь."
#         "\n\n"
#         "Контекст:"
#         f"{context}")

In [6]:
from db_config import db_config
import psycopg2
from langchain_community.chat_message_histories import SQLChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_postgres import PostgresChatMessageHistory
from langchain_ollama import ChatOllama
from langchain_core.runnables.history import RunnableWithMessageHistory

In [9]:
sync_connection = psycopg2.connect(**db_config)

# это только 1 раз
# table_name = "chat_history"
# PostgresChatMessageHistory.create_tables(sync_connection, table_name)

In [ ]:
from sqlalchemy.orm import sessionmaker, relationship, declarative_base
from sqlalchemy import create_engine, Column, Integer, String, Text, ForeignKey

# Define the SQLite database
DATABASE_URL = "postgresql+psycopg2://postgres:1@localhost:5432/habr"
Base = declarative_base()

# Define the Session model
class Session(Base):
    __tablename__ = "sessions"
    id = Column(Integer, primary_key=True)
    session_id = Column(String, unique=True, nullable=False)
    messages = relationship("Message", back_populates="session")

# Define the Message model
class Message(Base):
    __tablename__ = "messages"
    id = Column(Integer, primary_key=True)
    session_id = Column(Integer, ForeignKey("sessions.id"), nullable=False)
    role = Column(String, nullable=False)
    content = Column(Text, nullable=False)
    session = relationship("Session", back_populates="messages")

# Create the database and the tables
engine = create_engine(DATABASE_URL)
Base.metadata.create_all(engine)
SessionLocal = sessionmaker(bind=engine)

def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()


In [ ]:
from sqlalchemy.exc import SQLAlchemyError

from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

def load_session_history(session_id: str) -> BaseChatMessageHistory:
    db = next(get_db())
    chat_history = ChatMessageHistory()
    try:
        # Retrieve the session
        session = db.query(Session).filter(Session.session_id == session_id).first()
        if session:
            # Add each message to the chat history
            for message in session.messages:
                chat_history.add_message({"role": message.role, "content": message.content})
    except SQLAlchemyError:
        pass
    finally:
        db.close()

    return chat_history

In [ ]:
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = load_session_history(session_id)
    return store[session_id]


def invoke_and_save(session_id, input_text):
    # Save the user question with role "human"
    save_message(session_id, "human", input_text)
    
    # Get the AI response
    result = conversational_rag_chain.invoke(
        {"input": input_text},
        config={"configurable": {"session_id": session_id}}
    )["answer"]

    # Save the AI answer with role "ai"
    save_message(session_id, "ai", result)
    return result

In [7]:
from sqlalchemy import create_engine, Column, Integer, String, Text, ForeignKey
from sqlalchemy.orm import sessionmaker, relationship, declarative_base
from sqlalchemy.exc import SQLAlchemyError
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory

llm = ChatOllama(
    base_url="http://localhost:11434/",
    model="owl/t-lite",
    format="json",
    temperature = 0
)

# Define the SQLite database
DATABASE_URL = "postgresql+psycopg2://postgres:1@localhost:5432/habr"
Base = declarative_base()

class Session(Base):
    __tablename__ = "sessions"
    id = Column(Integer, primary_key=True)
    session_id = Column(String, unique=True, nullable=False)
    messages = relationship("Message", back_populates="session")

class Message(Base):
    __tablename__ = "messages"
    id = Column(Integer, primary_key=True)
    session_id = Column(Integer, ForeignKey("sessions.id"), nullable=False)
    role = Column(String, nullable=False)
    content = Column(Text, nullable=False)
    session = relationship("Session", back_populates="messages")

# Create the database and the tables
engine = create_engine(DATABASE_URL)
Base.metadata.create_all(engine)
SessionLocal = sessionmaker(bind=engine)

def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()

# Function to save a single message
def save_message(session_id: str, role: str, content: str):
    db = next(get_db())
    try:
        session = db.query(Session).filter(Session.session_id == session_id).first()
        if not session:
            session = Session(session_id=session_id)
            db.add(session)
            db.commit()
            db.refresh(session)

        db.add(Message(session_id=session.id, role=role, content=content))
        db.commit()
    except SQLAlchemyError:
        db.rollback()
    finally:
        db.close()

# Function to load chat history
def load_session_history(session_id: str) -> BaseChatMessageHistory:
    db = next(get_db())
    chat_history = ChatMessageHistory()
    try:
        session = db.query(Session).filter(Session.session_id == session_id).first()
        if session:
            for message in session.messages:
                chat_history.add_message({"role": message.role, "content": message.content})
    except SQLAlchemyError:
        pass
    finally:
        db.close()

    return chat_history

# Modify the get_session_history function to use the database
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = load_session_history(session_id)
    return store[session_id]

# Ensure you save the chat history to the database when needed
def save_all_sessions():
    for session_id, chat_history in store.items():
        for message in chat_history.messages:
            save_message(session_id, message["role"], message["content"])

# Example of saving all sessions before exiting the application
# import atexit
# atexit.register(save_all_sessions)


### Contextualize question ###
contextualize_q_system_prompt = """Учитывая историю чатов и последний вопрос пользователя, который может ссылаться на контекст в истории чата, \
                                сформулируй отдельный вопрос, который может быть понятбез истории чата. НЕ отвечайте на вопрос, только переформулируй его, если нужно, \
                                а в противном случае верни его как есть."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

### Answer question ###
qa_system_prompt = """Ты - помощник для решения задач, связанных с ответами на вопросы. Учитывая историю чатов и последний вопрос пользователя, \
                   который может ссылаться на контекст в истории чата, ответь на этот вопрос ИСПОЛЬЗУЯ фрагменты извлеченного контекста. \
                   Если ты не знаешь ответа, скажи, что ты не знаешь.
                   \n\n
                   Контекст:
                   {context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

### Statefully manage chat history ###
store = {}

conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

# Invoke the chain and save the messages after invocation
def invoke_and_save(session_id, input_text):
    # Save the user question with role "human"
    save_message(session_id, "human", input_text)
    
    result = conversational_rag_chain.invoke(
        {"input": input_text},
        config={"configurable": {"session_id": session_id}}
    )["answer"]

    # Save the AI answer with role "ai"
    save_message(session_id, "ai", result)
    return result

In [12]:
result = invoke_and_save("Swag2", "Расскажи мне про «Рокетбанк»")
print(result)

{ "content": "Рокетбанк — это российский онлайн-банк, который предоставляет услуги по управлению финансами и выпускает банковские карты. Основные особенности этого проекта включают в себя удобный интерфейс мобильного приложения, интуитивно понятное управление счетом и специальные функции для экономии средств и улучшенного управления личными финансами. Вот некоторые ключевые моменты о Рокетбанке, которые могут быть интересны пользователям и потенциальным клиентам:" }

  


In [13]:
result = invoke_and_save("Swag2", "Когда этот банк закрылся?")
print(result)

{ "content": "Рокетбанк прекратил свою деятельность в 2018 году. В октябре того года банк объявил о завершении всех операций и закрытии счетов клиентов, а также сообщил о продаже своих активов. Основные причины этого решения включали финансовые трудности и необходимость реорганизации после смены владельцев (в 2017 году он был приобретен банком «ФК Открытие»). После завершения всех процессов банк прекратил свою работу, а его клиенты были переведены в другие банки." }

  


In [16]:
system_prompt = (
"Учитывая историю чатов и последний вопрос пользователя "
"который может ссылаться на контекст в истории чата, "
"сформулируй отдельный вопрос, который может быть понят "
"без истории чата. НЕ отвечайте на вопрос, "
"только переформулируйте его, если нужно, а в противном случае верни его как есть."
)

connection_string = "postgresql+psycopg2://postgres:1@localhost:5432/habr"

def get_session_history(session_id):
    return SQLChatMessageHistory(session_id=session_id, table_name='history',session_id_field_name='session_id', connection_string=connection_string)

ru_llm = ChatOllama(
    base_url="http://localhost:11434/",
    model="owl/t-lite",
    format="json",
    temperature = 0.2
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)

runnable = prompt | ru_llm

runnable_with_history = RunnableWithMessageHistory(
    runnable,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

response = runnable_with_history.invoke(
    {"input": "Закрылся ли «Рокетбанк»?"},
    config={"configurable": {"session_id": "1"}},
)

TypeError: the JSON object must be str, bytes or bytearray, not NoneType